In [1]:
import pandas as pd
import glob
import re
import requests

import numpy as np

In [2]:
def mygender(name):
    lastname , firstname = name.split(",")
    firstname = firstname.split(" ")[0]
    #print(firstname)
    payload = {'name': str(firstname)}
    r = requests.get('https://api.genderize.io/?', params=payload)
    try:
        sex = r.json()['gender']
        if sex != None:
            return sex
        else:
            return 'male'
    except:
        return 'male'

In [3]:
class member:
    def __init__(self,name,ID,plan,level,sex):
        #print(name)
        self.lastname , self.firstname = name.split(",")
        self.ID = ID
        self.plan = plan
        self.level = level
        self.sex = sex
        
        
class groups(member):
    def __init__(self,code):
        self.number = 0
        self.member = []
        self.gplans = {}
        self.glevels = {}
        self.gsexs = {}
        self.code = code
        #self.Rec_sec = Rec_sec
        self.maxmember = 6
        
    def num(self):
        return self.number
    
    def add_member(self,mymember):
        if self.number < self.maxmember:
            self.member.append((mymember.firstname,mymember.lastname,mymember.ID))
            self.number += 1
            self.ID = mymember.ID

            if mymember.plan in self.gplans:
                self.gplans[mymember.plan] += 1
            else:
                self.gplans[mymember.plan] = 1


            if mymember.level in self.glevels:
                self.glevels[mymember.level] += 1
            else:
                self.glevels[mymember.level] = 1

            if mymember.sex in self.gsexs:
                self.gsexs[mymember.sex] += 1
            else:
                self.gsexs[mymember.sex] = 1

    def get_sex(self):
        return self.gsexs
        
    def get_levels(self):
        return self.glevels
        
    def get_plans(self):
        return self.gplans
    def get_members(self):
        return [self.member[i] for i in range(self.number)]
    
  

In [4]:
path = "./data/*.xlsx"

In [7]:
for file in glob.glob(path):
    
    dictGroups ={}
    lstMember =[]
    my_zip = {'70642':'114R1','70643':'114R2','70644':'114R3','70645':'114R4','70647':'114R5',
             '81063':'114R6','73464':'114R7','73465':'114R8','81064':'114R9','74823':'114R10','75922':'114R11',
             '76961':'114R12','76962':'114R13','78707':'114R14','79377':'114R15','82286':'114R16','84892':'114R17',
             '84893':'114R18','84894':'114R19','84895':'114R20','84896':'114R21'}
    my_table={0:"H",1:"He",2:"Li",3:"Be",4:"B",5:"C",6:"N",7:"O",8:"F",9:"Ne",10:"Na",11:"Mg",12:"Al"}
    a = file.split(" ")
    code = re.findall(r'\d+',a[0])[0]
    df = pd.read_excel(file,"Sheet1",header=None)
    df = df[[2,3,6,7]]
    df.columns = ["ID","Name","Program","Level"]
    df['Select'] = np.array(list(map(mygender,df.Name.tolist())))
    
    df = df.sort(["Select","Program","Level"])
    if df.shape[0] > 72:
        x = 13
    else:
        x = 12
    for i in range(x):
        dictGroups[i] = groups(code = code)
    
    for row in range(df.shape[0]):
        lstMember.append(member(df.iloc[row,1],df.iloc[row,0],df.iloc[row,2],df.iloc[row,3],df.iloc[row,4]))    
    
    for i in range(len(lstMember)):
        dictGroups[i%x].add_member(lstMember[i])
        
    b = {}
    for i in range(len(dictGroups)):
        b[i] = dictGroups[i].get_members()
        
    #pd.DataFrame(b).to_excel("output/"+code+"-"+my_zip[code]+".xlsx",sheet_name="Sheet1",index= False)    
    w = open("output/"+code+"-"+my_zip[code]+".csv","w")
    for item in b:
        for i in b[item]:
            firstname,lastname,ID = i
            w.write(my_table[item]+','+str(ID)+','+lastname+','+firstname+'\n')
    w.close()

C:\Users\skhezrse\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
